## Linear Regression 

Features(inputs): temperature, rainfall and humidity  
Targets(outputs): crop yeilds of apples and oranges

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1  
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2  
  
  



## Building model from scratch

In [38]:
import numpy as np
import torch

In [39]:
# Input (temp, rainfall, humidity)    3               784
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [40]:
# Targets (apples, oranges)           2               10
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [41]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [42]:
# Weights and biases
random_seed = 42
torch.manual_seed(random_seed)

w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863]], requires_grad=True)
tensor([ 2.2082, -0.6380], requires_grad=True)


In [43]:
def model(x):
    return x @ w.t() + b

In [44]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[  45.4987,  -67.0672],
        [  59.1878,  -90.4141],
        [  62.3595, -141.8688],
        [  50.7645,  -32.3210],
        [  54.2179, -105.5822]], grad_fn=<AddBackward0>)


In [45]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [46]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [47]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(19322.3477, grad_fn=<DivBackward0>)


In [48]:
# Compute gradients
loss.backward()

In [49]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863]], requires_grad=True)
tensor([[ -1622.2406,  -2731.8169,  -1496.6278],
        [-14781.0176, -17480.2109, -10474.4834]])


In [50]:
w
w.grad

tensor([[ -1622.2406,  -2731.8169,  -1496.6278],
        [-14781.0176, -17480.2109, -10474.4834]])

In [51]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [52]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(19322.3477, grad_fn=<DivBackward0>)


In [53]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [54]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ 49.1570, -40.0595],
        [ 64.0261, -54.8753],
        [ 68.2998, -99.5089],
        [ 54.1479,  -5.8505],
        [ 59.0077, -71.2684]], grad_fn=<AddBackward0>)


In [55]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(13438.3047, grad_fn=<DivBackward0>)


In [56]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ -1242.2931,  -2317.3828,  -1242.2360],
        [-11989.3076, -14471.1074,  -8619.6689]])
tensor([ -17.2723, -146.3125])


In [57]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [58]:
print(w)
print(b)

tensor([[ 0.3653,  0.1793,  0.2619],
        [ 0.4980, -0.8033,  0.0046]], requires_grad=True)
tensor([ 2.2086, -0.6347], requires_grad=True)


In [59]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(9467.9102, grad_fn=<DivBackward0>)


In [60]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [61]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(371.5295, grad_fn=<DivBackward0>)


In [62]:
# Predictions
preds

tensor([[ 62.1756,  77.6683],
        [ 83.4987, 103.6057],
        [107.7070, 114.4492],
        [ 47.8465,  80.1971],
        [ 88.9838,  98.9208]], grad_fn=<AddBackward0>)

In [63]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Linear regression using PyTorch built-ins

In [64]:
import torch

In [65]:
import torch.nn as nn
import numpy as np

In [66]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

In [67]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

In [68]:
# Convert the numpy array to a torch tensor.
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [69]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [70]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [71]:
from torch.utils.data import TensorDataset
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [72]:
from torch.utils.data import DataLoader
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [73]:
for xb, yb in train_dl:
    print(xb) #inputs
    print(yb) #targets
    break

tensor([[ 68.,  97.,  70.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 88., 134.,  59.],
        [ 74.,  66.,  43.]])
tensor([[102., 120.],
        [ 21.,  38.],
        [104., 118.],
        [118., 132.],
        [ 57.,  69.]])


## nn.Linear
Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [74]:
# Define model

model = nn.Linear(3, 2)
                #number of input vaiables, number of target
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2660, -0.0677, -0.2345],
        [ 0.3830, -0.4557, -0.2662]], requires_grad=True)
Parameter containing:
tensor([-0.1630, -0.3471], requires_grad=True)


PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [75]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.2660, -0.0677, -0.2345],
         [ 0.3830, -0.4557, -0.2662]], requires_grad=True),
 Parameter containing:
 tensor([-0.1630, -0.3471], requires_grad=True)]

In [76]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-34.2012, -14.3682],
        [-45.3356, -22.6343],
        [-45.9791, -43.5335],
        [-38.8834,   9.2736],
        [-41.4321, -36.3032],
        [-34.3995, -13.5295],
        [-45.5024, -22.4447],
        [-46.4796, -43.4166],
        [-38.6851,   8.4348],
        [-41.4006, -36.9523],
        [-34.3680, -14.1786],
        [-45.5339, -21.7956],
        [-45.8124, -43.7231],
        [-38.9150,   9.9227],
        [-41.2338, -37.1419]], grad_fn=<AddmmBackward0>)

In [77]:
# Loss Function
# Instead of defining a loss function manually, we can use the built-in loss function mse_loss.

# Import nn.functional
import torch.nn.functional as F
# The nn.functional package contains many useful loss functions and several other utilities.

# Define loss function
loss_fn = F.mse_loss

In [78]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(15413.5742, grad_fn=<MseLossBackward0>)


In [79]:
# Optimizer
# Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer 
# optim.SGD. SGD is short for "stochastic gradient descent". The term stochastic indicates that samples are 
# selected in random batches instead of as a single group.

# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

## Train the model
We are now ready to train the model. We'll follow the same process to implement gradient descent:

Generate predictions

Calculate the loss

Compute gradients w.r.t the weights and biases

Adjust the weights by subtracting a small quantity proportional to the gradient

Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function fit that trains the model for a given number of epochs.

In [80]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [81]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 978.2614
Epoch [20/100], Loss: 1119.3533
Epoch [30/100], Loss: 46.1012
Epoch [40/100], Loss: 86.2246
Epoch [50/100], Loss: 255.7762
Epoch [60/100], Loss: 248.1703
Epoch [70/100], Loss: 40.7621
Epoch [80/100], Loss: 56.8720
Epoch [90/100], Loss: 80.4346
Epoch [100/100], Loss: 39.8977


In [82]:
preds = model(inputs)
preds

tensor([[ 57.9632,  72.1837],
        [ 79.7249,  97.7961],
        [120.9140, 135.1310],
        [ 27.2696,  48.9016],
        [ 94.1150, 107.2309],
        [ 56.8129,  71.3099],
        [ 79.1278,  97.3103],
        [121.0001, 135.4895],
        [ 28.4200,  49.7753],
        [ 94.6681, 107.6188],
        [ 57.3660,  71.6979],
        [ 78.5746,  96.9224],
        [121.5112, 135.6168],
        [ 26.7165,  48.5137],
        [ 95.2653, 108.1046]], grad_fn=<AddmmBackward0>)

In [83]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [92]:
def compare_preds(pred, label):
    print("Prediction: ",pred[0].item(),"Label:",label[0].item())

In [93]:
compare_preds(preds[5],targets[5])

Prediction:  56.81285095214844 Label: 57.0
